In [14]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re
import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import operator

ov = cptac.Ovarian()
en = cptac.Endometrial()
co = cptac.Colon()

Checking that index is up-to-date...Loading clinical data...Loading cnv data...Loading definitions data...Loading phosphoproteomics data...Loading proteomics data...Loading somatic_38 data...Loading transcriptomics data...Loading treatment data...Formatting dataframes...Checking that index is up-to-date...Loading acetylproteomics data...Loading clinical data...Loading CNA data...Loading definitions data...Loading miRNA data...Loading phosphoproteomics_gene data...Loading phosphoproteomics_site data...Loading proteomics data...Loading somatic_binary data...Loading somatic data...Loading transcriptomics_circular data...Loading transcriptomics_linear data...Formatting dataframes...WARNING: This data is under a publication embargo until July 1, 2019.
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but analysis may not be
published until July 1, 2019. Please see
https:

In [2]:
def get_frequently_mutated(cancer_type, cutoff=.1):  
    """
    take cancer object and find the frequently 
    mutated genes in total tumors compared to the cutoff.
        
        Parameters:
        cancer_type (object): object of cancer class from cptac module 
        cutoff (float): used as comparison to determine status of gene mutation frequency
        
        Returns:
        freq_mutated_df (pd.DataFrame): DataFrame of frequently mutated genes passing the cutoff
            and Total_Mutated (mutated genes / total tumors), percent Truncated, and percent Missence
        
        There are many types of mutations catagorized into the columns Truncated and Missence. 
        The Truncated column includes: Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, Nonsense_Mutation, Nonstop_Mutation
        The Missence column includes: In_Frame_Del, In_Frame_Ins, Missense_Mutation
        These columns count multiple mutations of one gene in the same sample, so percentages in the last two columns may 
        exceed the Total_Mutated column(which only counts if the gene was mutated once)"""    
    
    # get data frames
    somatic_mutations = cancer_type.get_mutations()
    sample_status_map = cancer_type._get_sample_status_map()
    merged_mutations = somatic_mutations.join(sample_status_map, how="left") 
    """
    # standardize mutation names 
    if cancer_type.get_cancer_type() == 'Colon':
        mutation_equivalents = {'frameshift substitution': 'Frame_Shift_Del' , 'frameshift deletion': 'Frame_Shift_Del', 
            'frameshift insertion': 'Frame_Shift_Ins', 'stopgain': 'Nonsense_Mutation ', 'stoploss':'Nonstop_Mutation',
            'nonsynonymous SNV': 'Missense_Mutation','nonframeshift insertion': 'In_Frame_Ins',
            'nonframeshift deletion': 'In_Frame_Del', 'nonframeshift substitution': 'Missense_Mutation'}
        merged_mutations = merged_mutations.replace(to_replace = mutation_equivalents)
    """  
    # get list of unique genes
    unique_genes = somatic_mutations['Gene'].unique()
    
    # get total tumors/patients
    sample_status_series = sample_status_map.value_counts()
    total_tumor_patients = sample_status_series[0]
        
    # find frequently mutated genes and their total mutated fraction. Create lists for frequently mutated genes and fraction.
    freq_mut = [] 
    total_fraction_mutated = []
    for gene in unique_genes:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        gene_mutated = gene_mutated.index.unique()
        num_gene_mutated = len(gene_mutated)
        fraction = (num_gene_mutated / total_tumor_patients)
        if fraction > cutoff:
            freq_mut.append(gene)
            total_fraction_mutated.append(fraction)
    
    # find truncated fraction
    truncated = []
    for gene in freq_mut:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene] #possibly change var name
        #truncated_df = gene_mutated.loc[(gene_mutated['Mutation'] not in my_list)]
        truncated_df = gene_mutated.loc[(gene_mutated['Mutation'] != 'In_Frame_Del') & 
            (gene_mutated['Mutation'] != 'In_Frame_Ins') & (gene_mutated['Mutation'] != 'Missense_Mutation')] 
        samples_trunc = truncated_df.index.unique()
        num_trunc_mut = len(samples_trunc)
        fraction_trunc = (num_trunc_mut / total_tumor_patients)
        truncated.append(fraction_trunc)

    # find missence fraction #combine for loop
    missence = []
    for gene in freq_mut:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        #missence_mutations = gene_mutated.loc[(gene_mutated['Mutation'] in my_list)]
        missence_mutations = gene_mutated.loc[(gene_mutated['Mutation'] == 'In_Frame_Ins') | (gene_mutated['Mutation'] == 'In_Frame_Del') | (gene_mutated['Mutation'] == 'Missense_Mutation')]
        samples_miss = missence_mutations.index.unique()
        num_miss_mut = len(samples_miss)
        fraction_miss = (num_miss_mut / total_tumor_patients)
        missence.append(fraction_miss)
        
    # create dataframe
    merged_lists = list(zip(freq_mut, total_fraction_mutated, truncated, missence))
    freq_mutated_df = pd.DataFrame(merged_lists, columns =['Gene', 'Fraction_Mutated', 'Truncation', 'Missence'])
    freq_mutated_df.name = 'frequently_mutated'
                   
    return freq_mutated_df

In [37]:
fmo = get_frequently_mutated(ov, .2)
fme = get_frequently_mutated(en, .2)
fmc = get_frequently_mutated(co, .2)

In [38]:
genes = {}
for ind, row in fmc.iterrows():
    if row['Gene'] not in genes.keys():
        genes[row['Gene']] = [("COLON",row['Fraction_Mutated'])]
    else:
        genes[row['Gene']].append(("COLON",row['Fraction_Mutated']))
        

for ind, row in fme.iterrows():
    if row['Gene'] not in genes.keys():
        genes[row['Gene']] = [("ENDO",row['Fraction_Mutated'])]
    else:
        genes[row['Gene']].append(("ENDO",row['Fraction_Mutated']))
        
        
for ind, row in fmo.iterrows():
    if row['Gene'] not in genes.keys():
        genes[row['Gene']] = [("OVARIAN",row['Fraction_Mutated'])]
    else:
        genes[row['Gene']].append(("OVARIAN",row['Fraction_Mutated']))

        
for gene in genes.keys():
    print(gene, ": ", genes[gene])

APC :  [('COLON', 0.7454545454545455)]
TP53 :  [('COLON', 0.509090909090909), ('ENDO', 0.22105263157894736), ('OVARIAN', 0.6936936936936937)]
MUC16 :  [('COLON', 0.35454545454545455)]
FAT3 :  [('COLON', 0.2545454545454545)]
HMCN1 :  [('COLON', 0.20909090909090908)]
SYNE1 :  [('COLON', 0.2636363636363636)]
TTN :  [('COLON', 0.5454545454545454), ('OVARIAN', 0.27927927927927926)]
ARID1A :  [('COLON', 0.20909090909090908), ('ENDO', 0.45263157894736844)]
CSMD3 :  [('COLON', 0.23636363636363636)]
KRAS :  [('COLON', 0.3181818181818182), ('ENDO', 0.3263157894736842)]
TCF7L2 :  [('COLON', 0.21818181818181817)]
OBSCN :  [('COLON', 0.2636363636363636)]
MUC5B :  [('COLON', 0.24545454545454545)]
NCOR2 :  [('COLON', 0.21818181818181817)]
ZNF469 :  [('COLON', 0.22727272727272727)]
RYR1 :  [('COLON', 0.22727272727272727)]
PIK3CA :  [('COLON', 0.21818181818181817), ('ENDO', 0.49473684210526314)]
PLEC :  [('COLON', 0.20909090909090908)]
ACVR2A :  [('COLON', 0.22727272727272727)]
USF3 :  [('COLON', 0.218

In [39]:
fmo = get_frequently_mutated(ov)
fme = get_frequently_mutated(en)
fmc = get_frequently_mutated(co)

genes = {}
for ind, row in fmc.iterrows():
    if row['Gene'] not in genes.keys():
        genes[row['Gene']] = [("COLON",row['Fraction_Mutated'])]
    else:
        genes[row['Gene']].append(("COLON",row['Fraction_Mutated']))
        

for ind, row in fme.iterrows():
    if row['Gene'] not in genes.keys():
        genes[row['Gene']] = [("ENDO",row['Fraction_Mutated'])]
    else:
        genes[row['Gene']].append(("ENDO",row['Fraction_Mutated']))
        
        
for ind, row in fmo.iterrows():
    if row['Gene'] not in genes.keys():
        genes[row['Gene']] = [("OVARIAN",row['Fraction_Mutated'])]
    else:
        genes[row['Gene']].append(("OVARIAN",row['Fraction_Mutated']))

        
for gene in genes.keys():
    print(gene, ": ", genes[gene])

SCN2A :  [('COLON', 0.10909090909090909)]
AKAP13 :  [('COLON', 0.12727272727272726)]
ARHGEF5 :  [('COLON', 0.10909090909090909)]
APC :  [('COLON', 0.7454545454545455), ('ENDO', 0.1368421052631579)]
CDH23 :  [('COLON', 0.12727272727272726)]
NRXN1 :  [('COLON', 0.18181818181818182), ('ENDO', 0.12631578947368421)]
CADPS :  [('COLON', 0.10909090909090909), ('ENDO', 0.10526315789473684)]
FBXW7 :  [('COLON', 0.15454545454545454), ('ENDO', 0.18947368421052632)]
COL6A3 :  [('COLON', 0.10909090909090909), ('ENDO', 0.12631578947368421)]
TP53 :  [('COLON', 0.509090909090909), ('ENDO', 0.22105263157894736), ('OVARIAN', 0.6936936936936937)]
ADAMTS16 :  [('COLON', 0.12727272727272726)]
ADGRG4 :  [('COLON', 0.11818181818181818), ('ENDO', 0.1368421052631579)]
EVC2 :  [('COLON', 0.13636363636363635)]
PTPRZ1 :  [('COLON', 0.10909090909090909), ('ENDO', 0.1368421052631579)]
SMAD4 :  [('COLON', 0.11818181818181818)]
MUC16 :  [('COLON', 0.35454545454545455), ('ENDO', 0.18947368421052632), ('OVARIAN', 0.144

WDFY4 :  [('COLON', 0.10909090909090909), ('ENDO', 0.10526315789473684)]
PTPRS :  [('COLON', 0.12727272727272726)]
PCDH11X :  [('COLON', 0.13636363636363635)]
ASTN1 :  [('COLON', 0.12727272727272726), ('ENDO', 0.12631578947368421)]
PLEKHG4B :  [('COLON', 0.10909090909090909)]
NALCN :  [('COLON', 0.11818181818181818)]
MUC17 :  [('COLON', 0.14545454545454545), ('ENDO', 0.11578947368421053), ('OVARIAN', 0.12612612612612611)]
RP1 :  [('COLON', 0.11818181818181818), ('ENDO', 0.12631578947368421)]
TENM3 :  [('COLON', 0.16363636363636364), ('ENDO', 0.1368421052631579)]
JAKMIP3 :  [('COLON', 0.10909090909090909)]
TMEM132D :  [('COLON', 0.10909090909090909)]
PREX2 :  [('COLON', 0.10909090909090909)]
HYDIN :  [('COLON', 0.16363636363636364), ('ENDO', 0.10526315789473684)]
ZNF365 :  [('COLON', 0.12727272727272726)]
MYO18B :  [('COLON', 0.11818181818181818)]
PTPRF :  [('COLON', 0.11818181818181818)]
DNHD1 :  [('COLON', 0.16363636363636364)]
PRRC2C :  [('COLON', 0.11818181818181818)]
WDR87 :  [('CO